# Part 3: 

Students:
- Konstantinos Nikoletos 
- Konstantinos Plas

## Question 3.1: De-Duplication with Locality Sensitive Hashing

### Description
In this question you will be given a train set file with small texts. Every text is a Quora
question. You will also be given a test set in the same format. The purpose of this question
is to find how many of the documents in the test-set already exist in the train-set. As
duplicate we define document pairs with similarity more than a threshold τ=0.8. You have to
search for duplicate documents using the appropriate LSH family in order to reduce the time
required for the detection.

In [1]:
import pandas as pd
from tqdm import tqdm

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt

# nltk.download('stopwords')
# nltk.download('punkt')

## Reading the dataset

In [2]:
train_data = pd.read_csv('./Part-3/data/train_q_3.1.csv', names=['Question'],  delimiter='\t', header=0)
test_data = pd.read_csv('./Part-3/data/test_without_labels_q_3.1.csv',  delimiter='\t')

print(train_data.shape)
print(test_data.shape)

# train_data = train_data.head(100)
# test_data = test_data.head(100)

print("Train data shape: ", train_data.shape)
print(train_data.head())

print("\nTest data shape: ", test_data.shape)
print(test_data.head())

(3000, 1)
(1000, 1)
Train data shape:  (3000, 1)
                                            Question
0  What is the step by step guide to invest in sh...
1  What is the story of Kohinoor (Koh-i-Noor) Dia...
2  How can I increase the speed of my internet co...
3  Why am I mentally very lonely? How can I solve...
4  Which one dissolve in water quikly sugar, salt...

Test data shape:  (1000, 1)
                                            Question
0  What can someone do if they've lost the wirele...
1            Why India need to elect Prime minister?
2     How can I make money online with free of cost?
3  Does MDMA affect the first and higher order mo...
4  I am a Saudi National and have "SR 3 million" ...


In [3]:
train_data

,Question
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."
...,...
2995,What type of diets can you follow to lose 5 po...
2996,Which is the best commerce college in Mangalore?
2997,Is networking a good field to have a career in...
2998,Can I use letter stamps as postcard stamps?


## Pre-processing text

In [4]:
from stop_words import get_stop_words
stop_words_pypi = set(get_stop_words('en'))
# print(stop_words_pypi)

from nltk.corpus import stopwords
stop_words_nltk = set(stopwords.words('english'))
# print(stop_words_nltk)

manual_stop_words = {'include', 'way', 'work', 'look', 'add', 'time', 'year', 'one', \
                     'month', 'day', 'help', 'think', 'tell', 'new', 'said', 'say',\
                     'need', 'come', 'good', 'set', 'want', 'people', 'use', 'day', 'week', 'know'}

stop_words= stop_words_nltk.union(stop_words_pypi)
stop_words = stop_words.union(manual_stop_words)

In [5]:
# stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    processed_text = text.lower()
    processed_text = re.sub(r'\W', ' ', str(text))
    processed_text = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_text)
    processed_text = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_text)
    processed_text = re.sub(r'\s+', ' ', processed_text, flags=re.I)
    processed_text = re.sub(r'^b\s+', '', processed_text)

    tokens = [lemmatizer.lemmatize(word) for word in processed_text.split() if word not in stop_words]
    tokens = [token for token in tokens if token not in stop_words]
    processed_text = ' '.join(tokens)

    return processed_text

In [6]:
# from tqdm.notebook import tqdm
import os
from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

preprocessed_file_path_train = 'pre_train_31.csv'
if not os.path.exists(preprocessed_file_path_train):
    print("[TRAIN] Preprocessing text...")
    train_data['text'] = train_data['Question'].progress_apply(preprocess_text)
    print("[TRAIN] Preprocessing text done.")
    train_data.to_csv(preprocessed_file_path_train, index=False)
else:
    print("[TRAIN] Reading from file")
    train_data = pd.read_csv(preprocessed_file_path_train)

preprocessed_file_path_test = 'pre_test_31.csv'
if not os.path.exists(preprocessed_file_path_test):
    print("[TEST] Preprocessing text...")
    test_data['text'] = test_data['Question'].progress_apply(preprocess_text)
    print("[TEST] Preprocessing text done.")
    test_data.to_csv(preprocessed_file_path_test, index=False)
else:
    print("[TEST] Reading from file")
    test_data = pd.read_csv(preprocessed_file_path_test)

[TRAIN] Reading from file
[TEST] Reading from file


In [7]:
train_data['Question'][1]

'What is the story of Kohinoor (Koh-i-Noor) Diamond?'

In [8]:
train_data['text'][1]

'What story Kohinoor Koh Noor Diamond'

## Data vectorization

In [9]:
import time
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from datasketch import MinHashLSH, MinHash
import numpy as np

In [10]:
# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_data['text'])
X_test_tfidf = vectorizer.transform(test_data['text'])

vectorizer = CountVectorizer()
X_train_count = vectorizer.fit_transform(train_data['text'])
X_test_count = vectorizer.transform(test_data['text'])

In [11]:
# # Convert sparse matrices to dense arrays
# X_train_dense = X_train_tfidf.toarray()
# X_test_dense = X_test_tfidf.toarray()

## Cosine Similarity: Random projection LSH family.

## Exact Cosine

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
X_train = X_train_count
X_test = X_test_count

In [14]:
num_duplicates = 0

total_time = time.time()

Y = cosine_similarity(X_test_count, X_train_count)
num_duplicates += len(np.where((Y > 0.8).any(axis=1))[0])
# print(np.where((Y > 0.8).any(axis=1)))
print('Duplicates: {}'.format(num_duplicates))
print('Query time: {}'.format(time.time() - total_time))

Duplicates: 99
Query time: 0.04524850845336914


In [15]:
from scipy.spatial.distance import jaccard, cosine

In [16]:
from sklearn import random_projection

transformer = random_projection.GaussianRandomProjection(n_components=10)

X_new = transformer.fit_transform(X_train)
X_new = X_new > 0
better_hash = []
for c in X_new:
  better_hash.append(sum([j*(2**i) for i,j in list(enumerate(reversed(c)))]))
better_hash = np.array(better_hash)

train_data['hash'] = better_hash
hash_value = train_data['hash'].unique()

# print(hash_value)

num_duplicates = 0

true_duplicates = np.zeros((X_test.shape[0], X_train.shape[0]))

for hash_value in train_data['hash'].unique():
#     print(hash_value)
    xx = train_data[train_data['hash']==hash_value].index
#     print(xx)
    bucket = X_train[xx]
    for i in xx:
#         print( X_train[i])
        sim = cosine_similarity(X_test, X_train[i])
        z = np.where((sim > 0.8).any(axis=1))[0]
        for zz in z:
            true_duplicates[zz, i] = 1
        
#     print(Y.shape)
#     z = np.where((Y > 0.8).any(axis=1))[0]
#     print(z)
#     true_duplicates[]
#     true_d
#     num_duplicates += len(np.where((Y > 0.8).any(axis=1))[0])
#     true_knn =  NearestNeighbors(n_neighbors=2, algorithm='brute', metric='cosine').fit(bucket)
#     true_knn_distances, true_knn_indices = true_knn.kneighbors(X_test)
    
#     if an
    
#     print(true_knn_distances)
print('Duplicates: {}'.format(len(np.where((true_duplicates > 0).any(axis=1))[0])))

Duplicates: 99


In [17]:
X_new

array([[False,  True, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [ True,  True, False, ..., False,  True,  True],
       ...,
       [False,  True, False, ..., False, False,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ..., False,  True, False]])

In [18]:
from sklearn.neighbors import LSHForest

# # Function to calculate cosine similarity
# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Function to perform LSH with Random Projection
def lsh_cosine_similarity(train_features, test_features, threshold, k):
    # Build LSH model
    lshf = LSHForest(n_estimators=10, n_candidates=50, random_state=42, n_neighbors=5)
    build_start_time = time.time()
    lshf.fit(train_features)
    build_time = time.time() - build_start_time

    # Query LSH model
    query_start_time = time.time()
    distances, indices = lshf.kneighbors(test_features)
    query_time = time.time() - query_start_time

    # Calculate total time
    total_time = build_time + query_time

    # Count duplicates based on threshold
    num_duplicates = sum([1 for distance in distances if distance[0] > threshold])

    return build_time, query_time, total_time, num_duplicates


ImportError: cannot import name 'LSHForest' from 'sklearn.neighbors' (/home/nikoletos-ubuntu/.local/lib/python3.8/site-packages/sklearn/neighbors/__init__.py)

In [ ]:

# Loop through different values of K (1 to 10)
for k in range(1, 11):
    # Apply Gaussian Random Projection with n_components=10
    transformer = random_projection.GaussianRandomProjection(n_components=10, random_state=42)
    X_train_transformed = transformer.fit_transform(X_train)
    X_test_transformed = transformer.transform(X_test)

    # Set the threshold for similarity
    similarity_threshold = 0.8

    # Perform LSH with Cosine Similarity
    build_time, query_time, total_time, num_duplicates = lsh_cosine_similarity(X_train_transformed, X_test_transformed, similarity_threshold, k)

    # Print results in a table
    print(f"LSH-Cosine\t{build_time:.3f}\t{query_time:.3f}\t{total_time:.3f}\t{num_duplicates}\tK={k}")